In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df_maps = pd.read_csv('../data/maps_scores.csv')

# df_maps.dtypes
df_maps.head()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration
0,Champions Tour 2024: Americas Stage 2,Regular Season,Week 1,MIBR vs Leviatán,Ascent,MIBR,9,6,3,NaN,Leviatán,13,7,6,NaN,1:05:24
1,Champions Tour 2024: Americas Stage 2,Regular Season,Week 1,MIBR vs Leviatán,Icebox,MIBR,7,3,4,NaN,Leviatán,13,4,9,NaN,45:04
2,Champions Tour 2024: Americas Stage 2,Regular Season,Week 1,Sentinels vs NRG Esports,Lotus,Sentinels,13,8,5,NaN,NRG Esports,8,4,4,NaN,59:21
3,Champions Tour 2024: Americas Stage 2,Regular Season,Week 1,Sentinels vs NRG Esports,Sunset,Sentinels,14,9,3,2.0,NRG Esports,12,9,3,0.0,1:00:54
4,Champions Tour 2024: Americas Stage 2,Regular Season,Week 1,FURIA vs 100 Thieves,Icebox,FURIA,14,6,6,2.0,100 Thieves,12,6,6,0.0,1:12:51


In [3]:
# Lista de equipos participantes
participating_teams = ['FNATIC', 'Bilibili Gaming', 'DRX', 'KRÜ Esports', 'Gen.G', 'Sentinels', 'FunPlus Phoenix', 'Team Heretics', 'Leviatán', 'Talon Esports', 'Team Vitality', 'Trace Esports', 'EDward Gaming', 'FUT Esports', 'G2 Esports', 'Paper Rex']

# Eliminar columnas innecesarias
df_maps.drop(columns=['Tournament', 'Stage', 'Match Type', 'Match Name', 'Team A Attacker Score', 'Team A Defender Score', 'Team A Overtime Score', 'Team B Attacker Score', 'Team B Defender Score', 'Team B Overtime Score', 'Duration'], inplace=True)

In [4]:
# Filtrar los datos para mantener solo los de los equipos participantes
filtered_df_maps = df_maps[df_maps['Team A'].isin(participating_teams) | df_maps['Team B'].isin(participating_teams)]
filtered_df_maps.head(3)

,Map,Team A,Team A Score,Team B,Team B Score
0,Ascent,MIBR,9,Leviatán,13
1,Icebox,MIBR,7,Leviatán,13
2,Lotus,Sentinels,13,NRG Esports,8


In [5]:
# Crea una columna nueva con el ganador de cada mapa
filtered_df_maps['Map Winner'] = np.where(filtered_df_maps['Team A Score'] > filtered_df_maps['Team B Score'], filtered_df_maps['Team A'], filtered_df_maps['Team B'])
filtered_df_maps.head(3)

C:\Users\user\AppData\Local\Temp\ipykernel_9868\3021565895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_maps['Map Winner'] = np.where(filtered_df_maps['Team A Score'] > filtered_df_maps['Team B Score'], filtered_df_maps['Team A'], filtered_df_maps['Team B'])


,Map,Team A,Team A Score,Team B,Team B Score,Map Winner
0,Ascent,MIBR,9,Leviatán,13,Leviatán
1,Icebox,MIBR,7,Leviatán,13,Leviatán
2,Lotus,Sentinels,13,NRG Esports,8,Sentinels


In [6]:
# Crear tabla de victorias por mapa por equipo para este año
team_map_wins = filtered_df_maps.groupby(['Map Winner', 'Map']).size().unstack(fill_value=0)
team_map_wins = team_map_wins[team_map_wins.index.isin(participating_teams)]

team_map_wins.reset_index(inplace=True)
team_map_wins.rename(columns={'Map Winner': 'Team'}, inplace=True)

team_map_wins.head(3)

Map,Team,Abyss,Ascent,Bind,Breeze,Haven,Icebox,Lotus,Split,Sunset
0,Bilibili Gaming,0,3,4,0,0,5,5,1,3
1,DRX,1,5,5,3,4,4,3,3,2
2,EDward Gaming,1,4,8,6,2,4,2,6,8


In [7]:
# Crear tabla de cantidad de veces que se jugo cada mapa por equipo

team_map_counts = df_maps.groupby(['Team A', 'Map']).size().unstack(fill_value=0)
team_map_counts += df_maps.groupby(['Team B', 'Map']).size().unstack(fill_value=0)
team_map_counts = team_map_counts.loc[participating_teams]

team_map_counts.reset_index(inplace=True)
team_map_counts.rename(columns={'Map Winner': 'Team'}, inplace=True)

team_map_counts.head(3)

Map,Team A,Abyss,Ascent,Bind,Breeze,Haven,Icebox,Lotus,Split,Sunset
0,FNATIC,0,6,9,8,5,7,16,6,4
1,Bilibili Gaming,0,5,8,1,1,9,9,2,6
2,DRX,2,9,9,4,5,6,7,5,5


## Lista de Winrates por equipo por mapa

In [8]:
# Calcular el porcentaje de victorias por mapa por equipo
team_map_win_rates = ((team_map_wins.set_index('Team') / team_map_counts.set_index('Team A')) * 100).round(2)
team_map_win_rates.fillna(0, inplace=True)
# team_map_win_rates = team_map_win_rates.astype(str) + '%'
team_map_win_rates.reset_index(inplace=True)
team_map_win_rates

Map,index,Abyss,Ascent,Bind,Breeze,Haven,Icebox,Lotus,Split,Sunset
0,Bilibili Gaming,0.0,60.00,50.00,0.00,0.00,55.56,55.56,50.00,50.00
1,DRX,50.0,55.56,55.56,75.00,80.00,66.67,42.86,60.00,40.00
2,EDward Gaming,50.0,57.14,61.54,75.00,66.67,44.44,50.00,75.00,66.67
3,FNATIC,0.0,50.00,55.56,62.50,100.00,28.57,68.75,66.67,75.00
4,FUT Esports,0.0,100.00,44.44,50.00,0.00,50.00,54.55,60.00,41.67
5,FunPlus Phoenix,100.0,54.55,50.00,42.86,42.86,54.55,66.67,0.00,50.00
6,G2 Esports,100.0,70.00,61.54,50.00,50.00,42.86,62.50,62.50,33.33
7,Gen.G,0.0,82.35,66.67,66.67,66.67,62.50,76.19,50.00,50.00
8,KRÜ Esports,50.0,25.00,0.00,75.00,100.00,33.33,62.50,57.14,37.50
9,Leviatán,100.0,57.14,50.00,50.00,66.67,80.00,66.67,0.00,70.00


In [28]:
df_team_map_pick_rates = pd.read_csv("../data/draft_phase_mod/teams_maps_pick_rate.csv")
df_team_map_ban_rates = pd.read_csv("../data/draft_phase_mod/teams_maps_ban_rate.csv")

-----

In [29]:
map_list = df_maps['Map'].unique()
print(map_list)

['Ascent' 'Icebox' 'Lotus' 'Sunset' 'Haven' 'Bind' 'Split' 'Abyss'
 'Breeze']


In [30]:
for name_map in map_list:
    df_map = pd.DataFrame()


    df_map['Team'] = participating_teams
    df_map['Map'] = name_map
    df_map['Times Played'] = team_map_counts[name_map]
    df_map['Maps Won'] = df_map['Team'].map(team_map_wins.set_index('Team')[name_map])
    df_map['Maps Lost'] = df_map['Times Played'] - df_map['Maps Won']
    df_map['Map Win Rate (%)'] = df_map['Team'].map(team_map_win_rates.set_index('index')[name_map])

    for team in df_map['Team']:
        sum_team_score = df_maps.loc[(df_maps['Map'] == name_map) & (df_maps['Team A'] == team), 'Team A Score'].sum()
        sum_team_score += df_maps.loc[(df_maps['Map'] == name_map) & (df_maps['Team B'] == team), 'Team B Score'].sum()
        df_map.loc[df_map['Team'] == team, 'Rounds Won'] = sum_team_score
        df_map['Rounds Won'] = df_map['Rounds Won'].astype(int, errors='ignore')

        sum_team_rounds = df_maps.loc[(df_maps['Map'] == name_map) & (df_maps['Team A'] == team), 'Team B Score'].sum()
        sum_team_rounds += df_maps.loc[(df_maps['Map'] == name_map) & (df_maps['Team B'] == team), 'Team A Score'].sum()
        df_map.loc[df_map['Team'] == team, 'Rounds Lost'] = sum_team_rounds
        df_map['Rounds Lost'] = df_map['Rounds Lost'].astype(int, errors='ignore')

    df_map['Avg Round Win (Per Game)'] = (df_map['Rounds Won'] / (df_map['Times Played'])).round(2)
    df_map['Avg Round Loss (Per Game)'] = (df_map['Rounds Lost'] / (df_map['Times Played'])).round(2)

    df_map['Map Pick Rate (%)'] = df_map['Team'].map(df_team_map_pick_rates.set_index('Team')[name_map])
    df_map['Map Ban Rate (%)'] = df_map['Team'].map(df_team_map_ban_rates.set_index('Team')[name_map])
    df_map.fillna(0, inplace=True)
    df_map.rename(columns={'Times Played': 'times_played', 'Maps Won': 'maps_won', 'Maps Lost': 'maps_lost', 'Map Win Rate (%)': 'map_win_rate', 'Rounds Won': 'rounds_won', 'Rounds Lost': 'rounds_lost', 'Avg Round Win (Per Game)': 'avg_round_win_per_game', 'Avg Round Loss (Per Game)': 'avg_round_lost_per_game', 'Map Pick Rate (%)': 'map_pick_rate', 'Map Ban Rate (%)': 'map_ban_rate'}, inplace=True)
    df_map.to_csv(f'../data/team_stats/teams_{name_map}.csv', index=False)

In [31]:
df_map

,Team,Map,times_played,maps_won,maps_lost,map_win_rate,rounds_won,rounds_lost,avg_round_win_per_game,avg_round_lost_per_game,map_pick_rate,map_ban_rate
0,FNATIC,Breeze,8,5,3,62.50,85,81,10.62,10.12,19.23,0.00
1,Bilibili Gaming,Breeze,1,0,1,0.00,7,13,7.00,13.00,5.88,20.59
2,DRX,Breeze,4,3,1,75.00,48,29,12.00,7.25,4.76,5.71
3,KRÜ Esports,Breeze,4,3,1,75.00,51,41,12.75,10.25,10.53,14.29
4,Gen.G,Breeze,12,8,4,66.67,134,106,11.17,8.83,11.90,4.41
5,Sentinels,Breeze,4,0,4,0.00,26,52,6.50,13.00,0.00,25.00
6,FunPlus Phoenix,Breeze,7,3,4,42.86,75,82,10.71,11.71,16.67,4.17
7,Team Heretics,Breeze,5,2,3,40.00,49,47,9.80,9.40,0.00,13.56
8,Leviatán,Breeze,6,3,3,50.00,65,61,10.83,10.17,15.38,0.00
9,Talon Esports,Breeze,4,3,1,75.00,48,43,12.00,10.75,5.88,8.82
